# CM1 time and spatial averaging
Anna Mackie, 2022

Processing for long channel simulations from CM1. Three SSTs (295, 300 and 305K). Please see Wing et al. (2018) for simulation and variable descriptions.

This script, for different data:
- reads in data from CEDA archive
- takes the average over 24 hours and 32 x 32 grid points (equivilent to 96 km x 96km). NB last blocks may have more grid points to ensure all grid points used
- saves as npy files

This is done for a number of variables. The code is grouped for
1. 2D variables
2. 3D variables
3. Cloud fraction

Cloud fraction follows the method of Wing et al., (2020) which uses a threshold value for cloud condensate.

In [1]:
# activate virtual environmnet required for metpy
import sys
import pathlib
import platform
venv_path = '~/nb-venvs/metpy_venv'
sys.path.append(str(pathlib.Path(f'{venv_path}/lib/python{platform.python_version_tuple()[0]}.{platform.python_version_tuple()[1]}/site-packages/').expanduser()))

import metpy
print(metpy.__file__)

/home/users/arm33/nb-venvs/metpy_venv/lib/python3.10/site-packages/metpy/__init__.py


In [2]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from metpy.calc import saturation_mixing_ratio
from metpy.units import units
import numpy.ma as ma
import os
import sys
sys.path.append('../../')
import funcs



## Model specific inputs
Different models have slightly different set ups (eg grid points, file names etc)

'Blocks' refer to the grid post-spatial averaging

In [3]:
# model specific inputs
model = 'CM1' #in this model, time is in hours, with 24 hours to a file

#temp labels
temps =  ['large295', 'large300', 'large305']

In [4]:
# paramaters for all models
bk = 32 # number of x/y gridpoints in a block
nodays = 25 # number of days

#read in sample nc file to get dimensions
datapath3D = '/badc/rcemip/data/'+ model +'/RCE_' + temps[0]+ '/3D/'
nc_ta = Dataset(datapath3D + model + '_RCE_' + temps[0] + '_3D_allvars_hour1800.nc')
ta = nc_ta.variables['ta'][:]
tsize, levsize , ysize, xsize = np.shape(ta) 
nc_ta.close()
print(tsize, levsize, ysize, xsize)


1 74 134 2016


In [5]:
#set time, levels, y and x dimensions for this model
tsize, levsize, ysize, xsize = 1, 74, 134, 2016

## Parameters for all models

In [6]:
bk = 32 # number of x/y gridpoints in a block
nodays = 25 # number of days

In [7]:
# set up spaitial averaging
x_orig = np.arange(xsize) # total number of x points (rows)
y_orig = np.arange(ysize) # total number of y points (columns)

ny = int(len(y_orig)/bk) # number of x/y blocks in the grid
y_new = np.arange(ny*bk)
ybk = np.asarray(np.split(y_new, ny))

nx = int(len(x_orig)/bk) # number of x/y blocks in the grid
x_new = np.arange(nx*bk)
xbk = np.asarray(np.split(x_new, nx))
print('no blocks in x direction: ', nx, '; no in y direction: ',ny)


no blocks in x direction:  63 ; no in y direction:  4


## 2D data

- lwcrf - longwave cloud radiative effect, calculated from rlut (outgoing longwave radiation, all sky) and rlutcs (clear sky)
- swcrf - as above, but for shortwave
- pr - surface precipitation rate
- tas - near surface air temperature

In [8]:
#time params for 2D data
#set time params
ts = 24 # number of hours want to integrate over
nd = int(nodays*ts)
totalt = np.arange(nd)# total number of hours 
tbk = np.asarray(np.split(totalt, nodays))
print('takes the average over ' + str(nodays) + ' periods of ' + str(ts)+ ' hour averages')    

takes the average over 25 periods of 24 hour averages


In [9]:
dp = '/home/users/arm33/RCEMIP/'+ model + '/processed_new'

for temp in temps:

    datapath ='/badc/rcemip/data/'+ model + '/RCE_' + temp + '/2D/'
    datalist = sorted(os.listdir(datapath))
    datalist = datalist[-nd:] # last 25 days/600 hours
    d = len(datalist)
    print('total number of blocks: ', nx*ny*(d/24))

    lwcrf_bk, swcrf_bk = np.empty((nodays, ny, nx)),np.empty((nodays, ny, nx))
    pr_bk, tas_bk = np.empty((nodays, ny, nx)),np.empty((nodays, ny, nx))
    count = 0
    for t in range(0, d, ts):
        #empty arrays for each day
        lwcrf_day, swcrf_day = np.empty((ts, len(y_orig), len(x_orig) )),np.empty((ts, len(y_orig), len(x_orig) ))
        pr_day, tas_day = np.empty((ts, len(y_orig), len(x_orig) )), np.empty((ts, len(y_orig), len(x_orig) ))
        for tt in range(ts):
            fn = datapath + datalist[t+tt]
            nc = Dataset(fn)
            #read in data and fill in one day with  hourly data
            lwcrf_day[tt, :,:] = nc.variables['rlutcs'][:] - nc.variables['rlut'][:]
            swcrf_day[tt, :,:] = nc.variables['rsutcs'][:] - nc.variables['rsut'][:]
            pr_day[tt,:,:] = nc.variables['pr'][:]
            tas_day[tt,:,:]= nc.variables['tas'][:]
            nc.close()
        #take mean over that day and spatial blocks
        for i in range(ny):
                if i == ny-1: # check if it's the last block, if yes then use all remaining gridpoints
                    endy = y_orig[-1]+1
                else:
                    endy = ybk[i,-1]+1
                for j in range(nx): # ditto
                    if j == nx-1:
                        endx = x_orig[-1]+1
                    else:
                        endx = xbk[j,-1]+1

                    lwcrf_bk[count,i,j] = np.nanmean(lwcrf_day[:,  ybk[i,0]:endy, xbk[j,0]: endx])
                    swcrf_bk[count,i,j] = np.nanmean(swcrf_day[:,  ybk[i,0]:endy, xbk[j,0]: endx])
                    pr_bk[count,i,j] = np.nanmean(pr_day[:,  ybk[i,0]:endy, xbk[j,0]: endx])
                    tas_bk[count,i,j] = np.nanmean(tas_day[:,  ybk[i,0]:endy, xbk[j,0]: endx])

        if count%5==0:        
            print('day ', count, ' done')        
        count+=1

    lwcrf_bk.dump(dp + '/2D/'+ temp + 'lwcrf' + str(ts) + 'hrs.npy')
    swcrf_bk.dump(dp + '/2D/'+ temp + 'swcrf' + str(ts) + 'hrs.npy')
    pr_bk.dump(dp + '/2D/'+ temp + 'pr' + str(ts) + 'hrs.npy')
    tas_bk.dump(dp + '/2D/'+ temp + 'tas' + str(ts) + 'hrs.npy')

total number of blocks:  6300.0
day  0  done
day  5  done
day  10  done
day  15  done
total number of blocks:  6300.0
day  0  done
day  5  done
day  10  done
day  15  done
day  20  done
total number of blocks:  6300.0
day  0  done
day  5  done
day  10  done
day  15  done
day  20  done


## 3D data

Note that the 3D data is 6 hourly, so require different time parameters

- ua - eastward wind velocity
- va - northward wind velocity
- wa - vertical velocity
- pa - pressure
- cli - mass fraction of cloud liqid ice
- clw - mass fraction of cloud liquid water
- hus - specific humidity
- hur - relative humidity
- tntr - tendency of air temperature due to radiative heating
- ta - atmospheric temperature

In [6]:
#time params for 3D data
ts = 4 # number of timesteps want to integrate over
nd = int(nodays*ts)
totalt = np.arange(nd)# total number of timesteps 
tbk = np.asarray(np.split(totalt, nodays))
print('takes the average over ' + str(nodays) + ' periods of ' + str(ts)+ ' hour averages')    

takes the average over 25 periods of 4 hour averages


In [7]:
vars = ['ua', 'va', 'pa','cli', 'clw','wa','hus','hur', 'tntr', 'ta']

dp = '/home/users/arm33/RCEMIP/'+ model + '/processed_new'


for temp in temps:

    datapath ='/badc/rcemip/data/'+ model + '/RCE_' + temp + '/3D/'
    datalist = sorted(os.listdir(datapath))
    datalist = datalist[-nd:] # last 25 days/600 hours
    d = len(datalist)
    
    for var in vars:
        print(var)
        var_bk = np.empty((nodays,levsize ,ny, nx))
        count = 0
        for t in range(0, d, ts):
            #empty arrays for each day
            var_day = np.empty((ts, levsize,len(y_orig), len(x_orig) ))
            for tt in range(ts):
                fn = datapath + datalist[t+tt]
                nc = Dataset(fn)
                #read in data and fill in one day with  hourly data
                var_day[tt, :,:] = nc.variables[var][:]
                nc.close()
            #take mean over that day and spatial blocks
            for i in range(ny):
                    if i == ny-1: # check if it's the last block, if yes then use all remaining gridpoints
                        endy = y_orig[-1]+1
                    else:
                        endy = ybk[i,-1]+1
                    for j in range(nx): # ditto
                        if j == nx-1:
                            endx = x_orig[-1]+1
                        else:
                            endx = xbk[j,-1]+1
                        for k in range(levsize):
                            var_bk[count,k,i,j] = np.nanmean(var_day[:, k, ybk[i,0]:endy, xbk[j,0]: endx])
                        

            if count%5==0:        
                print('day ', count, ' done')        
            count+=1

        var_bk.dump(dp + '/3D/'+ temp + var + '_profile_25d.npy')
        



ua
day  0  done
day  5  done
day  10  done
day  15  done
day  20  done
va
day  0  done
day  5  done
day  10  done
day  15  done
day  20  done
ua
day  0  done
day  5  done
day  10  done
day  15  done
day  20  done
va
day  0  done
day  5  done
day  10  done
day  15  done
day  20  done
ua
day  0  done
day  5  done
day  10  done
day  15  done
day  20  done
va
day  0  done
day  5  done
day  10  done
day  15  done
day  20  done


## Cloud fraction

This follows the procedure described in Wing et al., 2020

1. Read in clw, cli ta and pa for each day (four time steps)
2. Take the mean for that 24 hours
3. Calculate the saturation mixing ratio and the cloud condensate (= cli + clw)
4. for each 32 x 32 block, calculate the proportion of points where either the cloud condensate is grater than 0.00001, or greater than the 0.01 x the saturation mixing ratio

In [9]:
#time params for 3D data
ts = 4 # number of timesteps want to integrate over
nd = int(nodays*ts)
totalt = np.arange(nd)# total number of timesteps 
tbk = np.asarray(np.split(totalt, nodays))
print('takes the average over ' + str(nodays) + ' periods of ' + str(ts)+ ' hour averages')    

takes the average over 25 periods of 4 hour averages


In [12]:
#cloud fraction
dp = '/home/users/arm33/RCEMIP/'+ model + '/processed_new'  
temps = ['large295']
nd = nodays*ts
for temp in temps:

    datapath = '/badc/rcemip/data/'+ model +'/RCE_' + temp + '/3D/'
    datalist = sorted(os.listdir(datapath))
    datalist = datalist[-nd:] # last 25 days
    d = len(datalist)

    satmixr =np.empty((nodays, levsize,len(y_orig), len(x_orig) ))
    cloudcon = np.empty((nodays, levsize,len(y_orig), len(x_orig) ))
    counter =0
    for t in np.arange(0,d,ts):
        clw_temp= np.empty((ts, levsize,len(y_orig), len(x_orig) ))
        cli_temp= np.empty((ts, levsize,len(y_orig), len(x_orig) ))
        pa_temp= np.empty((ts, levsize,len(y_orig), len(x_orig) ))
        ta_temp= np.empty((ts, levsize,len(y_orig), len(x_orig) ))
        for f in range(ts):
            fn = datapath + datalist[t+f]
            nc = Dataset(fn)
            clw_temp[f,] = nc.variables['clw'][:]
            cli_temp[f,] = nc.variables['cli'][:]
            pa_temp[f,] = nc.variables['pa'][:]
            ta_temp[f,] = nc.variables['ta'][:]
            nc.close()

        clwf = np.mean(clw_temp, axis = 0)
        clif = np.mean(cli_temp, axis = 0)
        paf = np.mean(pa_temp, axis = 0)
        taf = np.mean(ta_temp, axis = 0)

        pa = np.asarray(paf)*units.pascal
        ta = np.asarray(taf)*units.K
        
        satmixr[counter,  ] = metpy.calc.saturation_mixing_ratio(pa, ta)
        cloudcon[counter, ]= clwf + clif
        counter+=1
    print(np.shape(satmixr), np.shape(cloudcon))


    cldfrac = np.zeros((nodays, levsize, ny, nx))
    for t in range(nodays):
        print(t)
        for k in range(levsize):
            for i in range(ny):
                if i == ny-1: # check if it's the last block, if yes then use all remaining gridpoints
                    endy = y_orig[-1]+1
                else:
                    endy = ybk[i,-1]+1
                for j in range(nx): # ditto
                    if j == nx-1:
                        endx = x_orig[-1]+1
                    else:
                        endx = xbk[j,-1]+1

                    cc = cloudcon[t,k, ybk[i,0]:endy, xbk[j,0]: endx].flatten()
                    sm = satmixr[t,k, ybk[i,0]:endy, xbk[j,0]: endx].flatten()
                    no_grid_points = len(sm)
                    count = 0
                    for r in range(no_grid_points):
                        if cc[r] > 0.00001:
                            count +=1
                        elif cc[r] > 0.01*sm[r]:
                            count +=1 

                    cldfrac[t,k, i , j ] = count/no_grid_points

    cldfrac.dump(dp + '/3D/'+ temp + 'cldfrac_profile_25d.npy')

(25, 74, 134, 2016) (25, 74, 134, 2016)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
